In [1]:
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image

model_id = "Shashwath01/Idefic_medical_VQA_merged_4bit"

# Lấy processor từ mô hình gốc
processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics-9b")
model = AutoModelForImageTextToText.from_pretrained(model_id)


/home/d1l1th1um/Desktop/demo/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 7543.71it/s]
The 8-bit optimizer is not available on your device, only available on CUDA for now.


In [3]:
import torch
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

def do_inference(model, processor, image_path, question, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words_ids = tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids
    eos_token_id = tokenizer.eos_token_id  # Dùng eos_token mặc định

    img = Image.open(image_path)
    prompt = f"Question: {question}\nAnswer:"

    inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
    generated_ids = model.generate(
        **inputs,
        eos_token_id=eos_token_id,
        bad_words_ids=bad_words_ids,
        pad_token_id=tokenizer.pad_token_id,
        max_new_tokens=max_new_tokens
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)

do_inference(model, processor, "image.jpg", "Please provide a detailed step-by-step explanation of the important medical findings in this X-ray image, including possible diagnoses, anatomical structures involved, and any recommendations for further tests or treatments.", max_new_tokens=50)


Question: Please provide a detailed step-by-step explanation of the important medical findings in this X-ray image, including possible diagnoses, anatomical structures involved, and any recommendations for further tests or treatments.
Answer:

This is a chest X-ray of a 10-year-old boy with a history of recurrent pneumonia. The patient is on a ventilator.

The most striking finding is the large pleural
